# Practical Exam: Spectrum Shades LLC
Spectrum Shades LLC is a prominent supplier of concrete color solutions, offering a wide range of pigments and coloring systems used in various concrete applications, including decorative concrete, precast concrete, and concrete pavers. The company prides itself on delivering high-quality colorants that meet the unique needs of its diverse clientele, including contractors, architects, and construction companies.
</br></br>
The company has recently observed a growing number of customer complaints regarding inconsistent color quality in their products. The discrepancies have led to a decline in customer satisfaction and a potential increase in product returns.
By identifying and mitigating the factors causing color variations, the company can enhance product reliability, reduce customer complaints, and minimize return rates.
</br></br>
You are part of the data analysis team tasked with providing actionable insights to help Spectrum Shades LLC address the issues of inconsistent color quality and improve customer satisfaction.

# Task 1

Before you can start any analysis, you need to confirm that the data is accurate and reflects what you expect to see. 

It is known that there are some issues with the `production_data` table, and the data team have provided the following data description. 

Write a query to ensure the data matches the description provided, including identifying and cleaning all invalid values. You must match all column names and description criteria.
</br>

- You should start with the data in the file "production_data.csv".
- Your output should be a DataFrame named clean_data.
- All column names and values should match the table below.
</br>

| Column Name             | Criteria                                                                                         |
|--------------------------|--------------------------------------------------------------------------------------------------|
| batch_id | Discrete. Identifier for each batch. Missing values are not possible. |
| production_date | Date. Date when the batch was produced.|
| raw_material_supplier | Categorical. Supplier of the raw materials. (1='national_supplier', 2='international_supplier'). <br> Missing values should be replaced with 'national_supplier'.|
| pigment_type           | Nominal. Type of pigment used. ['type_a', 'type_b', 'type_c']. <br> Missing values should be replaced with 'other'. |
| pigment_quantity       | Continuous. Amount of pigment added (in kilograms) (Range: 1 - 100). <br> Missing values should be replaced with median. |
| mixing_time           | Continuous. Duration of the mixing process (in minutes). <br> Missing values should be replaced with mean. |
| mixing_speed          | Categorical. Speed of the mixing process represented as categories: 'Low', 'Medium', 'High'.</br> Missing values should be replaced with 'Not Specified'. |
| product_quality_score | Continuous. Overall quality score of the final product (rating on a scale of 1 to 10). <br> Missing values should be replaced with mean. |


In [130]:
# Write your answer to Task 1 here

import pandas as pd 
data = pd.read_csv('production_data.csv')
clean_data = data.copy()

missing_values = ['-', 'missing', 'N/A', 'NA', 'None']
clean_data.replace(missing_values, pd.NA, inplace=True)

clean_data['batch_id'] = clean_data['batch_id'].astype('category')

clean_data['production_date'] = pd.to_datetime(clean_data['production_date'], errors='coerce')
clean_data = clean_data.dropna(subset=['production_date'])

clean_data['raw_material_supplier'] = clean_data['raw_material_supplier'].astype('category')
clean_data['raw_material_supplier'].fillna('national_supplier', inplace=True)
valid_suppliers = ['national_supplier', 'international_supplier']
clean_data['raw_material_supplier'] = clean_data['raw_material_supplier'].apply(
    lambda x: x if x in valid_suppliers else 'national_supplier')

clean_data['pigment_type'] = pd.Categorical(clean_data['pigment_type'])
clean_data['pigment_type'].replace({'Type_C': 'type_c'}, inplace=True)
clean_data['pigment_type'].fillna('other', inplace=True)
valid_pigments = ['type_a', 'type_b', 'type_c']
clean_data['pigment_type'] = clean_data['pigment_type'].apply(
    lambda x: x if x in valid_pigments else 'other')

pigment_median = clean_data['pigment_quantity'].median()
clean_data['pigment_quantity'].fillna(pigment_median, inplace=True)

clean_data['mixing_time'].fillna(clean_data['mixing_time'].mean(), inplace=True)

clean_data["mixing_speed"] = clean_data["mixing_speed"].astype('category')
clean_data['mixing_speed'] = clean_data['mixing_speed'].cat.add_categories('Not Specified')
clean_data['mixing_speed'].fillna('Not Specified', inplace=True)

quality_mean = clean_data['product_quality_score'].mean()
clean_data['product_quality_score'].fillna(quality_mean, inplace=True)

print(clean_data)



     batch_id production_date  ... mixing_speed product_quality_score
0           1      2024-06-25  ...         High              7.165102
1           2      2023-11-23  ...         High              6.849126
2           3      2024-02-18  ...         High              5.661209
3           4      2023-11-11  ...         High              6.991735
4           5      2024-04-11  ...         High              7.095043
...       ...             ...  ...          ...                   ...
1995     1996      2024-03-10  ...          Low              5.043882
1996     1997      2024-05-10  ...          Low              7.584934
1997     1998      2024-05-21  ...       Medium              6.178186
1998     1999      2024-02-15  ...          Low              5.055166
1999     2000      2024-04-10  ...         High              9.477531

[2000 rows x 8 columns]


# Task 2

You want to understand how the supplier type and quantity of materials affect the final product attributes.

Calculate the average `product_quality_score` and `pigment_quantity` grouped by `raw_material_supplier`.

- You should start with the data in the file 'production_data.csv'. 
- Your output should be a DataFrame named aggregated_data.
- It should include the three columns: `raw_material_supplier`, `avg_product_quality_score`, and `avg_pigment_quantity`.
- Your answers should be rounded to 2 decimal places.


In [131]:
# Write your answer to Task 2 here

import pandas as pd 
data = pd.read_csv('production_data.csv')
df = data.copy()

# Group the data by 'raw_material_supplier' and calculate the averages
aggregated_data = df.groupby('raw_material_supplier')[['product_quality_score', 'pigment_quantity']].mean()

# Round the averages to 2 decimal places
aggregated_data = aggregated_data.round(2)

# Reset the index to include 'raw_material_supplier' as a column
aggregated_data = aggregated_data.reset_index()

# Rename the columns as required
aggregated_data.columns = ['raw_material_supplier', 'avg_product_quality_score', 'avg_pigment_quantity']

# Display the resulting DataFrame
print(aggregated_data)

   raw_material_supplier  avg_product_quality_score  avg_pigment_quantity
0                      1                       8.02                 44.73
1                      2                       5.97                 34.91


# Task 3

Identify the average `product_quality_score` for batches with a `raw_material_supplier` of 2 and a `pigment_quantity` greater than 35 kg.

Write a query to return the average `avg_product_quality_score` for these filtered batches. Use the original production data table, not the output of Task 2.

- You should start with the data in the file 'production_data.csv'. 
- Your output should be a DataFrame named pigment_data.
- It should include the columns: `raw_material_supplier`, `pigment_quantity`, and `avg_product_quality_score`.
- Your answers should be rounded to 2 decimal places.


In [132]:
import pandas as pd

# Load the production data from the CSV file
production_data = pd.read_csv('production_data.csv')

# Step 1: Filter the data for raw_material_supplier = 2 and pigment_quantity > 35
filtered_data = production_data[(production_data['raw_material_supplier'] == 2) & (production_data['pigment_quantity'] > 35)]

# Step 2: Calculate the average product_quality_score for the filtered batches
average_quality_score = filtered_data['product_quality_score'].mean()

# Step 3: Prepare the result DataFrame
pigment_data = filtered_data[['raw_material_supplier', 'pigment_quantity']].drop_duplicates()
pigment_data['avg_product_quality_score'] = average_quality_score

# Step 4: Round the avg_product_quality_score to 2 decimal places
pigment_data = pigment_data.round(2)

# Step 5: Display the result
print(pigment_data)


      raw_material_supplier  pigment_quantity  avg_product_quality_score
1                         2             42.87                       5.97
4                         2             36.21                       5.97
6                         2             35.94                       5.97
7                         2             40.50                       5.97
8                         2             36.02                       5.97
...                     ...               ...                        ...
1985                      2             40.48                       5.97
1986                      2             41.98                       5.97
1994                      2             37.48                       5.97
1995                      2             42.92                       5.97
1998                      2             46.15                       5.97

[619 rows x 3 columns]


# Task 4

In order to proceed with further analysis later, you need to analyze how various factors relate to product quality. Start by calculating the mean and standard deviation for the following columns: `pigment_quantity`, and `product_quality_score`. </br> These statistics will help in understanding the central tendency and variability of the data related to product quality.
</br> </br >
Next, calculate the Pearson correlation coefficient between the following variables: `pigment_quantity`, and `product_quality_score`.
</br>
These correlation coefficients will provide insights into the strength and direction of the relationships between the factors and overall product quality.


- You should start with the data in the file 'production_data.csv'.
- Calculate the mean and standard deviation for the columns pigment_quantity and product_quality_score as: `product_quality_score_mean`, `product_quality_score_sd`, `pigment_quantity_mean`, `pigment_quantity_sd`.
- Calculate the Pearson correlation coefficient between pigment_quantity and product_quality_score as: `corr_coef`
- Your output should be a DataFrame named product_quality.
- It should include the columns: `product_quality_score_mean`, `product_quality_score_sd`, `pigment_quantity_mean`, `pigment_quantity_sd`, `corr_coef`.
- Ensure that your answers are rounded to 2 decimal places.


In [133]:
# Write your answer to Task 4 here

import pandas as pd

# Load the data from the CSV file
df = pd.read_csv('production_data.csv')

# Calculate the mean and standard deviation for product_quality_score and pigment_quantity
product_quality_score_mean = df['product_quality_score'].mean()
product_quality_score_sd = df['product_quality_score'].std()
pigment_quantity_mean = df['pigment_quantity'].mean()
pigment_quantity_sd = df['pigment_quantity'].std()

# Calculate the Pearson correlation coefficient between pigment_quantity and product_quality_score
corr_coef = df['pigment_quantity'].corr(df['product_quality_score'])

# Create the result DataFrame with the required columns
product_quality = pd.DataFrame({
    'product_quality_score_mean': [round(product_quality_score_mean, 2)],
    'product_quality_score_sd': [round(product_quality_score_sd, 2)],
    'pigment_quantity_mean': [round(pigment_quantity_mean, 2)],
    'pigment_quantity_sd': [round(pigment_quantity_sd, 2)],
    'corr_coef': [round(corr_coef, 2)]
})

# Display the result
print(product_quality)


   product_quality_score_mean  ...  corr_coef
0                        6.68  ...       0.49

[1 rows x 5 columns]
